# Image Classification with Traditional Methods

### Extract HOG features for training and testing data

In [ ]:
import numpy as np
from sklearn.neighbors import KNeighborsClassifier
from skimage.feature import hog
from skimage import color
from sklearn.metrics import accuracy_score
from tensorflow.keras.datasets import cifar10

# Load CIFAR-10 dataset
(x_train, y_train), (x_test, y_test) = cifar10.load_data()

# Define HOG feature extraction function
def extract_hog_features(images):
    hog_features = []
    for img in images:
        img_gray = color.rgb2gray(img)  # Convert to grayscale
        features = hog(img_gray, pixels_per_cell=(8, 8),
                        cells_per_block=(2, 2), block_norm='L2-Hys', visualize=False)
        hog_features.append(features)
    return np.array(hog_features)


print("Extracting HOG features for training data...")
X_train_hog = extract_hog_features(x_train)
print("Extracting HOG features for testing data...")
X_test_hog = extract_hog_features(x_test)


### Initialize a list of arrays for each class (0 to 9)

In [ ]:

num_classes = 10
hog_features_by_class = [np.array([]).reshape(0, X_train_hog.shape[1]) for _ in range(num_classes)]
hog_features_by_class_test = [np.array([]).reshape(0, X_test_hog.shape[1]) for _ in range(num_classes)]

# Populate the lists with HOG features
for i in range(num_classes):
    # Select indices where the label is i
    indices = np.where(y_train.flatten() == i)[0]
    indices0 = np.where(y_test.flatten() == i)[0]
    # Append the corresponding HOG features
    hog_features_by_class[i] = X_train_hog[indices]
    hog_features_by_class_test[i] = X_test_hog[indices0]

# Check the shapes of the arrays for each class
for i in range(num_classes):
    print(f"Class {i} has {hog_features_by_class_test[i].shape[0]} samples with feature dimension {hog_features_by_class[i].shape[1]}")

# Example: Access HOG features for class 0
hog_features_class_0 = hog_features_by_class[0]
print("Shape of HOG features for class 0:", hog_features_class_0.shape)


Class 0 has 1000 samples with feature dimension 324
Class 1 has 1000 samples with feature dimension 324
Class 2 has 1000 samples with feature dimension 324
Class 3 has 1000 samples with feature dimension 324
Class 4 has 1000 samples with feature dimension 324
Class 5 has 1000 samples with feature dimension 324
Class 6 has 1000 samples with feature dimension 324
Class 7 has 1000 samples with feature dimension 324
Class 8 has 1000 samples with feature dimension 324
Class 9 has 1000 samples with feature dimension 324
Shape of HOG features for class 0: (5000, 324)


### Collaborative Representation Classifier

In [ ]:
# Training phase
subjects = list(range(10))  # Subjects 0 to 9
sample_per_subject = 5000
lambda_reg = 0.1

dict_list = np.empty((0, 324))
subadd = np.zeros(10, dtype=int)
for j in subjects:
    features=hog_features_by_class[j][:sample_per_subject]
    dict_list = np.vstack((dict_list, features))  # Use vstack to stack vertically
    subadd[j] = dict_list.shape[0]

dict_matrix = np.vstack(dict_list)
n_sample, num_features = dict_matrix.shape
P = np.linalg.inv(dict_matrix.T @ dict_matrix + lambda_reg * np.eye(num_features)) @ dict_matrix.T

In [ ]:
# Testing phase
test_samples = 300
taux = []
P0=P.T
for n in range(test_samples):
    similarity_matrix = np.zeros((num_classes, num_classes))
    for j in subjects:
        D1=hog_features_by_class_test[j][n]
        x0 = P0 @ D1;
        res = np.zeros((1, num_classes))
        b = 0
        for c in range(num_classes):
            l_x0 = np.zeros((n_sample))
            l_x0[b:subadd[c]] = x0[b:subadd[c]]
            res[0, c] = np.sum(( l_x0 @ dict_matrix - D1) ** 2)
            b = subadd[c]

        similarity_matrix[j, :] = res


    fiin = np.argmin(similarity_matrix, axis=0)
    ttt = np.zeros(similarity_matrix.shape[1])
    for qqq in range(similarity_matrix.shape[1]):
        if fiin[qqq] == qqq:
            ttt[qqq] = 1
        else:
            ttt[qqq] = 0

    taux.append(np.sum(ttt) / num_classes * 100)

mean_accuracy = np.mean(taux)
print("Mean Accuracy:", mean_accuracy)

Mean Accuracy: 52.792792792792795


# K-NN classifier

In [ ]:
# Train K-NN classifier
print("Training 1-NN classifier...")
knn = KNeighborsClassifier(n_neighbors=100)
knn.fit(X_train_hog, y_train.ravel())

# Predict on the test set
print("Predicting on test data...")
y_pred = knn.predict(X_test_hog)

# Evaluate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy*100:.4f}")

Training 1-NN classifier...
Predicting on test data...
Accuracy: 45.9600
